In [101]:
import torch
import torchvision
import torch.nn as nn
from torchsummary import summary
import warnings
from torch.nn.modules.utils import _pair
import import_ipynb
from AnchorGeneratorWhichWorks import iou_threshold

In [102]:
class conv(nn.Module):
    
    def __init__(self, c, s):
    
        super().__init__()
        
        if s==1:
            self.seq = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=s,padding=1,kernel_size=3),nn.BatchNorm2d(num_features=c),nn.SiLU())
        if s==2:
            self.seq = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=2*c,stride=s,padding=1,kernel_size=3),nn.BatchNorm2d(num_features=2*c),nn.SiLU())
    
    def forward(self, x):
        
        x = self.seq(x)
        
        return x

In [103]:
class MPconv(nn.Module):
    
    def __init__(self,c):
        
        super().__init__()
        
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=1)
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=2*c,stride=2,kernel_size=1),nn.BatchNorm2d(num_features=2*c),nn.SiLU())
        self.conv2 = conv(c,2)
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=4*c,out_channels=2*c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=2*c),nn.SiLU())
        
    def forward(self,x):
        
        xa = self.maxpool(x)
        xa = self.conv1(xa)
        
        xb = self.conv3(x)
        xb = self.conv2(xb)
        
        x = torch.concat((xa,xb),dim=1)
        
        x = self.conv4(x)
        
        return x

In [104]:
class REPconv(nn.Module):
    
    def __init__(self,c):
        
        super().__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.conv2 = conv(c,1)
        self.bn = nn.BatchNorm2d(c)
    
    def forward(self,x):
        
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.bn(x)
        
        if self.training:
            x = torch.add(x1,x2)
            x = torch.add(x,x3)
            return x
        else:
            return x1

In [105]:
class ELAN1(nn.Module):
    
    def __init__(self,c):
        
        super().__init__()
        
        self.convELEN1a = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.convELEN1b = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.convELEN1c = conv(c,1)
        self.convELEN1d = conv(c,1)
        self.convELEN1e = conv(c,1)
        self.convELEN1f = conv(c,1)
        self.convELEN1g = nn.Sequential(nn.Conv2d(in_channels=3*c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        
    def forward(self,x):
        
        x1 = self.convELEN1a(x)
        x2 = self.convELEN1b(x)
        x3 = self.convELEN1c(x1)
        x3 = self.convELEN1d(x3)
        x4 = self.convELEN1e(x3)
        x4 = self.convELEN1f(x4)
        
        x = torch.cat((x2,x3,x4),dim=1)
        
        x = self.convELEN1g(x)
        
        return x

In [106]:
class ELAN2(nn.Module):
    
    def __init__(self,c):
        
        super().__init__()
        
        self.convELEN1a = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.convELEN1b = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.convELEN1c = conv(c,1)
        self.convELEN1d = conv(c,1)
        self.convELEN1e = conv(c,1)
        self.convELEN1f = conv(c,1)
        self.convELEN1g = nn.Sequential(nn.Conv2d(in_channels=5*c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        
    def forward(self,x):
        
        x1 = self.convELEN1a(x)
        x2 = self.convELEN1b(x)
        x3 = self.convELEN1c(x1)
        x4 = self.convELEN1d(x3)
        x5 = self.convELEN1e(x4)
        x6 = self.convELEN1f(x5)
        
        x = torch.concat((x2,x3,x4,x5,x6),dim=1)
        
        x = self.convELEN1g(x)
        
        return x

In [107]:
class SPPCSPC(nn.Module):
    
    def __init__(self,c):
        
        super().__init__()
        
        self.conv1a = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.conv1b = conv(c,1)
        self.conv1c = nn.Sequential(nn.Conv2d(in_channels=c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.maxpool1a = nn.MaxPool2d(kernel_size=5,padding=2)
        self.maxpool1b = nn.MaxPool2d(kernel_size=9,padding=4)
        self.maxpool1c = nn.MaxPool2d(kernel_size=13,padding=6)
        self.conv1d = nn.Sequential(nn.Conv2d(in_channels=4*c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        self.conv1e = conv(c,1)
        self.conv1f = nn.Sequential(nn.Conv2d(in_channels=2*c,out_channels=c,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=c),nn.SiLU())
        
    def forward(self,x):
        
        x1a = self.conv1a(x)
        x2 = self.conv2(x)
        
        x1b = self.conv1b(x1a)
        x1c = self.conv1c(x1b)
        x1ma = self.maxpool1a(x1c)
        sfa = (x1c.shape[2]/x1ma.shape[2])
        x1ma = nn.UpsamplingBilinear2d(scale_factor=sfa)(x1ma)
        x1mb = self.maxpool1b(x1c)
        sfb = (x1c.shape[2]/x1mb.shape[2])
        x1mb = nn.UpsamplingBilinear2d(scale_factor=sfb)(x1mb)
        x1mc = self.maxpool1c(x1c)
        sfc = (x1c.shape[2]/x1mc.shape[2])
        x1mc = nn.UpsamplingBilinear2d(scale_factor=sfc)(x1mc)
        x1c = torch.cat((x1c,x1ma,x1mb,x1mc),dim=1)
        
        x1d = self.conv1d(x1c)
        x1e = self.conv1e(x1d)
        x = torch.cat((x1e,x2),dim=1)
        
        x = self.conv1f(x)
        
        return x

In [108]:
class Backbone(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=32,stride=1,kernel_size=3,padding=1),nn.BatchNorm2d(num_features=32),nn.SiLU())
        self.conv2 = conv(32,2)         #This will give output of 64 channels
        self.conv3 = conv(64,1)
        self.conv4 = conv(64,2)         #This will give output of 128 channels
        self.ELAN1a = ELAN1(128)
        self.MPconv1 = MPconv(128)      #This will give output of 256 channels
        self.ELAN1b = ELAN1(256)
        self.MPconv2 = MPconv(256)      #This will give output of 512 channels
        self.ELAN1c = ELAN1(512)
        self.MPconv3 = MPconv(512)      #This will give output of 1024 channels
        self.ELAN1d = ELAN1(1024)
        self.SPPCSPC = SPPCSPC(1024)
        
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.ELAN1a(x)
        x = self.MPconv1(x)
        x1 = self.ELAN1b(x)
        x2 = self.MPconv2(x1)
        x2 = self.ELAN1c(x2)
        x3 = self.MPconv3(x2)
        x3 = self.ELAN1d(x3)
        x3 = self.SPPCSPC(x3)
        
        return (x1,x2,x3)

In [109]:
class head(nn.Module):
    
    def __init__(self,no_of_classes,img_size,ground_truth_boxes):
        
        super().__init__()
        
        self.backbone = Backbone()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1024,out_channels=512,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=512),nn.SiLU())
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=512,out_channels=512,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=512),nn.SiLU())
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=256,out_channels=256,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=256),nn.SiLU())
        self.upsample1 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.convCat1 = nn.Sequential(nn.Conv2d(in_channels=1024,out_channels=512,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=512),nn.SiLU())
        self.ELAN2a = ELAN2(512)
        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=512,out_channels=256,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=256),nn.SiLU())
        self.upsample2 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.convCat2 = nn.Sequential(nn.Conv2d(in_channels=768,out_channels=256,stride=1,kernel_size=1),nn.BatchNorm2d(num_features=256),nn.SiLU())
        
        self.ELAN2b = ELAN2(256)
        self.MPconv1 = MPconv(256) # The output for this is 512 channels
        self.ELAN2c = ELAN2(512)
        self.MPconv2 = MPconv(512) # The output for this is 1024 channels
        self.ELAN2d = ELAN2(1024)
        
        self.conv5 = nn.Conv2d(in_channels=256,out_channels=(no_of_classes+5)*3,stride=1,kernel_size=1)
        self.avgpool1 = nn.AvgPool2d((int(img_size/8),int(img_size/8)))
        self.head1 = nn.Softmax(1)
        
        self.conv6 = nn.Conv2d(in_channels=512,out_channels=(no_of_classes+5)*3,stride=1,kernel_size=1)
        self.avgpool2 = nn.AvgPool2d((int(img_size/16),int(img_size/16)))
        self.head2 = nn.Softmax(1)
        
        self.conv7 = nn.Conv2d(in_channels=1024,out_channels=(no_of_classes+5)*3,stride=1,kernel_size=1)
        self.avgpool3 = nn.AvgPool2d(kernel_size=(int(img_size/32),int(img_size/32)))
        self.head3 = nn.Softmax(1)
        
        self.gtb = ground_truth_boxes
        
    def forward(self,x):
        
        y = self.backbone(x)
        x1 = self.conv1(y[2])
        x1 = self.upsample1(x1)
        x1 = torch.cat((x1,y[1]),dim=1)
        x1 = self.convCat1(x1)
        x1 = self.ELAN2a(x1)
        
        x2 = self.conv2(x1)
        x2 = self.upsample2(x2)
        x2 = torch.cat((x2,y[0]),dim=1)
        x2 = self.convCat2(x2)
        x2 = self.ELAN2b(x2)
        
        head1 = self.conv5(x2)
        if self.training:
            anchors1,scores1 = iou_threshold(head1,self.gtb/8)
        head1 = self.avgpool1(head1)
        head1 = self.head1(head1)
        
        x3 = self.MPconv1(x2)
        x3 = torch.cat((x1,x3),dim=1)
        x3 = nn.Conv2d(in_channels=1024,out_channels=512,kernel_size=1)(x3)
        x3 = self.ELAN2c(x3)
        
        head2 = self.conv6(x3)
        if self.training:
            anchors2,scores2 = iou_threshold(head2,self.gtb/16)
        head2 = self.avgpool2(head2)
        head2 = self.head2(head2)
        
        x4 = self.MPconv2(x3)
        x4 = torch.cat((x4,y[2]),dim=1)
        x4 = nn.Conv2d(in_channels=2048,out_channels=1024,kernel_size=1)(x4)
        x4 = self.ELAN2d(x4)
        
        head3 = self.conv7(x4)
        if self.training:
            anchors3,scores3 = iou_threshold(head3,self.gtb/32)
        head3 = self.avgpool3(head3)
        head3 = self.head3(head3)
        
        if self.training:
            return ((head1,(anchors1,scores1)), (head2,(anchors2,scores2)), (head3,(anchors3,scores3)))
        else:
            return (head1 ,head2, head3)

In [110]:
import torch
torch.cuda.empty_cache()

In [111]:
b = torch.rand((4,5))
b.shape

torch.Size([4, 5])

In [116]:
a = torch.rand((4,3,640,640))
b = torch.tensor([[54,54,40,40]])
model = head(20,640,b)(a)
model.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [117]:
model[2][0].shape

torch.Size([4, 75, 1, 1])

In [120]:
model[2][1]

([], [])